# Проект для «Викишоп»

**Задача**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. Нужно обучить модель классифицировать комментарии на позитивные и негативные. Имеется набор данных с разметкой о токсичности правок.

Метрики качества *F1* должна быть не меньше 0.75. 

**План работы:**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [2]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv(r"C:\Users\Markm\Downloads\toxic_comments.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
#возбмем выборку из 50000 строк
df = df.sample(50000).reset_index(drop =True)

In [6]:
#удалим неинформативный столбец
df =df.drop('Unnamed: 0', axis = 1)

In [7]:
text = list(df['text'])

In [8]:
#напишем функцию для очистки текста (с помощью регулярного выражения)
def clear_text(text):
    text_cleared = []
    for elem in text:
        text = re.sub(r'[^a-zA-Z]', ' ', str(elem))
        text = text.split()
        text_cleared. append(" ".join(text).lower())
    return text_cleared

In [9]:
text_cleared = clear_text(text)

In [10]:
nlp = spacy.load('en_core_web_lg')

In [11]:
#напишем функцию для лемматизации текста, с помощью библиотеки spacy
def lemmatize(text):
    lemmatize_text = []
    for elem in text:
        doc = nlp(elem)
        token = ' '.join([token.lemma_ for token in doc])
        lemmatize_text.append(token)
    return lemmatize_text

In [12]:
lem_text = lemmatize(text_cleared)

In [13]:
df['text'] = pd.Series(lem_text)

In [14]:
text_col = ['text']

**Выводы**
1. Данные очищены и лемматизированы.

# Обучение

In [15]:
X_series = df['text']
X_dataframe = df[['text']]
 
y = df['toxic']

In [16]:
features_train, features_test, target_train, target_test = train_test_split(X_series,y, test_size = 0.5, random_state = 42, stratify = y)

In [17]:
features_test.shape

(25000,)

In [18]:
target_test.shape

(25000,)

In [19]:
#создадим пайплайн с TFidVectorizer и логистической регрессией
model_pipe = Pipeline(
    [
        (
            'vect',
            TfidfVectorizer()
        ),
       
        (
            'clf',
            LogisticRegression(
                random_state=42
            )
        )
    ])
 
 
model_pipe.fit(features_train.head(100), target_train.head(100))

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clf', LogisticRegression(random_state=42))])

In [20]:
#Создадим пространство гиперпараметров для логистической регрессии
params = [{
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'clf': [LogisticRegression(random_state=42)],
    'clf__C' : [0.1, 10, 20],
    'clf__class_weight': [None, 'balanced']
    }
]

In [21]:
grid = GridSearchCV(
        model_pipe,
        params,
        cv=4,
        n_jobs=-1,
        scoring='f1',
        error_score='raise',
    )

In [22]:
grid.fit(features_train, target_train)
 
display(grid)
display(grid.best_estimator_)
print(grid.best_params_)
print(grid.best_score_)

GridSearchCV(cv=4, error_score='raise',
             estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'clf': [LogisticRegression(C=20,
                                                     class_weight='balanced',
                                                     random_state=42)],
                          'clf__C': [0.1, 10, 20],
                          'clf__class_weight': [None, 'balanced'],
                          'vect__ngram_range': [(1, 1), (1, 2), (2, 2)]}],
             scoring='f1')

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clf',
                 LogisticRegression(C=20, class_weight='balanced',
                                    random_state=42))])

{'clf': LogisticRegression(C=20, class_weight='balanced', random_state=42), 'clf__C': 20, 'clf__class_weight': 'balanced', 'vect__ngram_range': (1, 1)}
0.7455599245289822


In [23]:
pred_log = grid.best_estimator_.predict(features_test)

In [24]:
f1 = f1_score(target_test, pred_log)
f1

0.7587405833494302

Попробуем модель CatBoostClassifier.

In [25]:
model = CatBoostClassifier(random_state = 42, task_type = 'CPU', verbose = False)

In [26]:
features_test = features_test.to_numpy()

In [27]:
#Создадим обучающие и тестовые пулы
train_data = Pool(features_train.to_numpy(), target_train, text_features =list(range(len(text_col))))
test_data = Pool(features_test, text_features =list(range(len(text_col))))

In [28]:
model.fit(train_data)

In [29]:
preds_class = model.predict(test_data)

In [30]:
preds_class

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [31]:
f1 = f1_score(target_test, preds_class)
f1

0.7649933362949801

# Общий вывод

1. Обучено две модели: LogisticRegression (в пайплайне с TfidfVectorizer())  и CatBoostClassifier.
2. Параметры LogisticRegression - C=10, class_weight='balanced', random_state=42, TfidVectorizer - ngram_range: (1, 1). F1 score - 0.7553 на тестовых данных.
3. f1 score у CatBoostClassifier без настройки гиперпараметров - 0.7649.
4. Обе модели удовлетворяют условиям задачи, однако у CatBoostClassifier даже без настройки гиперпараметров метрика немного лучше.